In [ ]:
%run ../00_AdvancedPythonConcepts/talktools.py

# Sockets and Ports

An Internet socket is an endpoint of a bidirectional inter-process communication flow across an Internet Protocol-based computer network, such as the Internet. The term Internet sockets is also used as a name for an application programming interface (API) for the TCP/IP protocol stack, usually provided by the operating system. Internet sockets constitute a mechanism for delivering incoming data packets to the appropriate application process or thread, based on a combination of local and remote IP addresses and port numbers. Each socket is mapped by the operating system to a communicating application process or thread.

A port is an application-specific or process-specific software construct serving as a communications endpoint.

-- wikipedia

# Socket

Provides access to the BSD socket interface 
using TCP or UDP. Great for communication to any IP address
(internal, LAN, or external).

Functions:
 - socket() -- create a new socket object
 - socketpair() -- create a pair of new socket objects [*]
 - fromfd() -- create a socket object from an open file descriptor [*]
 - gethostname() -- return the current hostname
 - gethostbyname() -- map a hostname to its IP number
 - gethostbyaddr() -- map an IP number or hostname to DNS info
 - getservbyname() -- map a service name and a protocol name to a port number
 - getprotobyname() -- map a protocol name (e.g. 'tcp') to a number
 - ntohs(), ntohl() -- convert 16, 32 bit int from network to host byte order
 - htons(), htonl() -- convert 16, 32 bit int from host to network byte order
 - inet_aton() -- convert IP addr string (123.45.67.89) to 32-bit packed format
 - inet_ntoa() -- convert 32-bit packed format IP to string (123.45.67.89)
 - ssl() -- secure socket layer support (only available if configured)
 - socket.getdefaulttimeout() -- get the default timeout value
 - socket.setdefaulttimeout() -- set the default timeout value
 - create_connection() -- connects to an address, with an optional timeout

 [*] not available on all platforms!

# TCP Socket Server

In [ ]:
import socket, time
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
host, port = "", 5100
s.bind((host, port))
s.listen(1)
conn, addr = s.accept()
print("Client is at", addr, ".")
data = conn.recv(100)
print("Client says:", data)
response = "Invalid command."
if data == "hello":
    response = "How do you do?"
if data == "time":
    response = time.ctime()
if data == "goodbye":
    response = "It was nice hearing from you."
conn.send(response) # now send response
conn.close()

Make a server that runs indefinitely. Must kill it with an interrupt.

In [3]:
import socket, socketserver, time
class EchoRequestHandler(socketserver.BaseRequestHandler):
    def handle(self):
        data = self.request.recv(100)
        response = "Invalid command."
        if data == "hello":
            response = "How do you do?"
        if data == "time":
            response = time.ctime()
        self.request.send(response)
server = socketserver.ThreadingTCPServer(('', 5011), EchoRequestHandler)
try:
    server.serve_forever()
except KeyboardInterrupt:
    del server
    print("you interrupted. bye.")

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 57450)
----------------------------------------


Traceback (most recent call last):
  File "/Users/MattGilbert/anaconda/lib/python3.5/socketserver.py", line 625, in process_request_thread
    self.finish_request(request, client_address)
  File "/Users/MattGilbert/anaconda/lib/python3.5/socketserver.py", line 354, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/MattGilbert/anaconda/lib/python3.5/socketserver.py", line 681, in __init__
    self.handle()
  File "<ipython-input-3-11ec5cbe5131>", line 10, in handle
    self.request.send(response)
TypeError: a bytes-like object is required, not 'str'


you interrupted. bye.


# XML-RPC Overview

 - Remote Procedure Call protocol which uses XML to encode its calls and HTTP as a transport mechanism
 - A Client sends an HTTP request to a Server implementing the protocol
 - The Client can pass multiple input parameters to the Server and request that the Server perform a method on those parameters and return the result (one value) in the response
 - The parameters and result can be common data types (including lists of multiple values)
 - Data is translated through XML (Extensible Markup Language) for transmission
 - In our usage the data is reformed into Python data structures by modules at the Client and Server

XML-RPC Server:

In [ ]:
try:
    from xmlrpc.server import SimpleXMLRPCServer
except:
    # Python 2.7
    import SimpleXMLRPCServer

class Some_Class_We_Want_Remotely_Accessible(object):
    def addition(self, a, b):
        return a + b
    def subtraction(self, a, b):
        return a - b

host, port = "", 5022
server = SimpleXMLRPCServer((host, port), allow_none=True)
server.register_instance(Some_Class_We_Want_Remotely_Accessible())
server.register_multicall_functions()
server.register_introspection_functions()
print("XMLRPC Server is starting at:", host, port)

In [ ]:
server.serve_forever()